In [1]:
import pandas as pd
import warnings
import math
from matplotlib import pyplot as plt
from datetime import datetime
import sys
import numpy as np

# Ignora todos os warnings
warnings.filterwarnings('ignore')

In [2]:
print("Hora:", datetime.now().strftime("%H:%M:%S"))

Hora: 15:14:17


In [9]:
#LISTA DE HQ
df_tel_hq=pd.read_excel('C:/Users/abner/OneDrive/Documentos/mestrado/DISSERTAÇÃO/scripts/Estacoes_telemetria_HQ.xlsx')

In [27]:
# general path
general_path = 'C:/Users/abner/OneDrive/Documentos/mestrado/Telemetria-ANA/Scripts e dados'

states = ['AC','AL','AM','AP','BA','CE','DF','ES','GO','MA','MG','MS','MT','PA','PB','PE','PI','PR','RJ','RN','RO','RR','RS','SC','SE','SP','TO']

years = list(range(2014,2025))
mit_int=360
method = "RUSLE2"   # "USLE", "RUSLE" or "RUSLE2"

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [29]:
def calculate_KE(I, method):
    if method.upper() == "USLE":
        if I <= 76:
            return 0.119 + 0.0873 * math.log10(I)
        else:
            return 0.283
    elif method.upper() == "RUSLE":
            return 0.29 * (1 - 0.72 * math.exp(-0.05 * I))
    elif method.upper() == "RUSLE2":
        return 0.29 * (1 - 0.72 * math.exp(-0.082 * I))

In [35]:
for state in states:
    for year in years:
        start_time = datetime.now()
        name_col=state+ '_ '+ str(year)
        mit=pd.Timedelta(mit_int, unit ='m')
        name_file= state+ '_'+ str(year)+'.h5'

        df_info = pd.read_hdf(general_path +'/1 - Organized data/'+ name_file,'table_info')
        df_year_hq=df_tel_hq[year].dropna().astype(int)
        common_values = set(df_info['gauge_code'].astype(int)).intersection(set(df_year_hq))
        hq_name_file=list(map(str, common_values))
        df_rainfall = pd.read_hdf(general_path +'/1 - Organized data/'+ name_file,'table_data').dropna(subset=['rain_mm']).reset_index(drop=True)
        print(f"Initialize : {state}_{year} {start_time.strftime("%Y-%m-%d %H:%M:%S")}") 


        monthly_results = {m: {"gauge_code": [], "EI30": [], "E_D": [], "rain": []} for m in range(1, 13)}
        annual_results = {"gauge_code": [], "EI30": [], "E_D": [], "rain": []}
        for code in hq_name_file:
            start_time_code = datetime.now()
            name=code +'_' + str(mit_int)
            df_rainfall_events = pd.read_hdf(general_path + '/3 - Rainfall events/'+ name_file, name)
            df_partial_result = pd.DataFrame(columns=['rain_mm','duration', 'I30', 'RF_KE', 'EI30'])
            for evento in range(0,len(df_rainfall_events)):

                beg=df_rainfall_events.iloc[evento,1]
                end=df_rainfall_events.iloc[evento,1]+pd.Timedelta(hours=df_rainfall_events.iloc[evento,3])
                month = beg.month
                df_gauge_rainfall=df_rainfall[df_rainfall['gauge_code']==code]
                df_event = df_gauge_rainfall[(df_gauge_rainfall['datetime'] >= beg) & (df_gauge_rainfall['datetime'] <= end)].reset_index(drop=True)

                if df_event['rain_mm'].sum()>12.7:  #SPECIFICATION AT RUSLE 2
                    for i in df_event.index:
                        RF=df_event['rain_mm'][i] #mm
                        time_delta=int((df_event.iloc[i,3]-df_event.iloc[i-1,3]).total_seconds() // 60)
                        if i==0:RF_intensity=(RF/30)*60 #mm/h  ###tdel
                        elif time_delta==0:RF_intensity=(RF/15)*60 #mm/h  ###tdel
                        else:RF_intensity=(RF/time_delta)*60 #mm/h  ###tdel
                        #kinectic energy
                        KE = calculate_KE(RF_intensity, method) if RF != 0 else 0
                        RF_KE=KE*RF#MJ/ha
                        if (KE > 0.283) :KE= 0.283 #max value 
                        
                        #calculate I30
                        tbeg=df_event.iloc[i,3] 
                        tend=df_event.iloc[i,3]+pd.Timedelta(hours=0.5)
                        df_temp=df_event[(df_event['datetime'] >= tbeg) & (df_event['datetime'] < tend)]#### < ou <=
                        I30=df_temp['rain_mm'].sum()/0.5 #mm/h #intervalo de 0,5 hrs

                        df_event.at[i,'Rf_intensity']=RF_intensity
                        df_event.at[i,'RF_KE']=RF_KE
                        df_event.at[i,'I30']=I30

                    #calc ei30       
                    EI30=df_event['RF_KE'].sum()*df_event['I30'].max()        
                    #set partial results
                    df_partial_result.at[evento,'rain_mm']=df_event['rain_mm'].sum() 
                    df_partial_result.at[evento,'duration']=df_rainfall_events.iloc[evento,3]        
                    df_partial_result.at[evento,'I30']=df_event['I30'].max() 
                    df_partial_result.at[evento,'RF_KE']=df_event['RF_KE'].sum()
                    df_partial_result.at[evento,'EI30']=EI30
                    df_partial_result.at[evento,'month']=month
                
                else:

                    df_partial_result.at[evento,'rain_mm']=df_rainfall_events.iloc[evento,2]  
                    df_partial_result.at[evento,'duration']=df_rainfall_events.iloc[evento,3]        
                    df_partial_result.at[evento,'I30']=0
                    df_partial_result.at[evento,'RF_KE']=0
                    df_partial_result.at[evento,'EI30']=0
                    df_partial_result.at[evento,'month']=month
                #set final results        
            with pd.HDFStore(general_path + f'/1 - Results_Telemetria/Results_{state}_{year}.h5', 'a') as store:
                for month in range(1, 13):
                #set monthly results
                    df_month = df_partial_result[df_partial_result['month']==month]
                    EI30_M=df_month['EI30'].sum()
                    rain=df_month['rain_mm'].sum()
                    E_D = EI30_M / rain if rain != 0 else float('nan')
                    monthly_results[month]["gauge_code"].append(code)
                    monthly_results[month]["EI30"].append(EI30_M)
                    monthly_results[month]["E_D"].append(E_D)
                    monthly_results[month]["rain"].append(rain)                
                    
                    store.put(f'month_{month:02d}',pd.DataFrame(monthly_results[month]))
            
                #set anual results
                
                EI30_T=df_partial_result['EI30'].sum()
                E_D=EI30_T/df_partial_result['rain_mm'].sum()
                rain=df_partial_result['rain_mm'].sum()
                annual_results["gauge_code"].append(code)
                annual_results["EI30"].append(EI30_T)
                annual_results["E_D"].append(E_D)
                annual_results["rain"].append(rain)
                store.put('annual', pd.DataFrame(annual_results))


            print(f"  -code: {code} , time: {(datetime.now()-start_time_code).total_seconds():.2f}s , {len(annual_results['E_D'])}/{len(hq_name_file)}")

        minutes, seconds = divmod((datetime.now()-start_time).total_seconds(), 60)#total duration
        print(f"{state}_{year} completed, total duration: {int(minutes)}m {seconds:.2f}s")  

Initialize : AC_2014 2025-03-28 16:53:50
  -code: 12640000 , time: 10.95s , 1/7
  -code: 13568000 , time: 9.05s , 2/7
  -code: 13540000 , time: 8.67s , 3/7
  -code: 12510500 , time: 10.62s , 4/7
  -code: 13405000 , time: 11.48s , 5/7
  -code: 12590000 , time: 10.67s , 6/7
  -code: 13450000 , time: 9.15s , 7/7
AC_2014 completed, total duration: 1m 11.35s
Initialize : AC_2015 2025-03-28 16:55:02
  -code: 13600002 , time: 9.50s , 1/5
  -code: 12510500 , time: 15.88s , 2/5
  -code: 13405000 , time: 12.28s , 3/5
  -code: 13578000 , time: 10.67s , 4/5
  -code: 12590000 , time: 11.45s , 5/5
AC_2015 completed, total duration: 1m 2.03s
Initialize : AC_2016 2025-03-28 16:56:04
  -code: 12640000 , time: 15.03s , 1/6
  -code: 13540000 , time: 8.60s , 2/6
  -code: 12510500 , time: 10.20s , 3/6
  -code: 13310000 , time: 10.20s , 4/6
  -code: 12590000 , time: 9.70s , 5/6
  -code: 13470000 , time: 8.93s , 6/6
AC_2016 completed, total duration: 1m 8.07s
Initialize : AC_2017 2025-03-28 16:57:12
  -code:

KeyboardInterrupt: 

In [ ]:
for state in states:
    for year in years:
        start_time = datetime.now()
        mit=pd.Timedelta(mit_int, unit ='m')
        name_file= state+ '_'+ str(year)+'.h5'
        name_col= state+ '_ '+ str(year)
        print(f"Initialize : {state}_{year} {start_time.strftime("%Y-%m-%d %H:%M:%S")}") 
        df_rainfall = pd.read_hdf(general_path +'/1 - Organized data/'+ name_file,'table_data')

        monthly_results = {m: {"gauge_code": [], "EI30": [], "E_D": [], "rain": []} for m in range(1, 13)}
        annual_results = {"gauge_code": [], "EI30": [], "E_D": [], "rain": []}
        for code in df_tel_hq[name_col].dropna():
            start_time_code = datetime.now()  
            name=code +'_' + str(mit_int)
            df_rainfall_events = pd.read_hdf(general_path + '/3 - Rainfall events/'+ name_file, name).drop_duplicates()
            df_partial_result = pd.DataFrame(columns=['rain_mm','duration', 'I30', 'RF_KE', 'EI30'])
            for evento in range(0,len(df_rainfall_events)):
                beg=df_rainfall_events.iloc[evento,1]
                month = beg.month
                end=df_rainfall_events.iloc[evento,1]+pd.Timedelta(hours=df_rainfall_events.iloc[evento,3])
                df_gauge_rainfall=df_rainfall[df_rainfall['gauge_code']==code]
                df_event = df_gauge_rainfall[(df_gauge_rainfall['datetime'] >= beg) & (df_gauge_rainfall['datetime'] <= end)].reset_index(drop=True)

                if df_event['rain_mm'].sum()>12.7:  #SPECIFICATION AT RUSLE 2
                    for i in df_event.index:
                        RF=df_event['rain_mm'][i] #mm
                        RF_intensity=(RF/10)*60 #mm/h  ###tdel
                        #kinectic energy
                        KE = calculate_KE(RF_intensity, method) if RF != 0 else 0
                        RF_KE=KE*RF#MJ/ha
                        if (KE > 0.283) :KE= 0.283 #max value 
                        
                        #calculate I30
                        tbeg=df_event.iloc[i,3] 
                        tend=df_event.iloc[i,3]+pd.Timedelta(hours=0.5)
                        df_temp=df_event[(df_event['datetime'] >= tbeg) & (df_event['datetime'] < tend)]#### < ou <=
                        I30=df_temp['rain_mm'].sum()/0.5 #mm/h #intervalo de 0,5 hrs

                        df_event.at[i,'Rf_intensity']=RF_intensity
                        df_event.at[i,'RF_KE']=RF_KE
                        df_event.at[i,'I30']=I30

                    #calc ei30       
                    EI30=df_event['RF_KE'].sum()*df_event['I30'].max()        
                    #set partial results
                    df_partial_result.at[evento,'rain_mm']=df_event['rain_mm'].sum() 
                    df_partial_result.at[evento,'duration']=df_rainfall_events.iloc[evento,3]        
                    df_partial_result.at[evento,'I30']=df_event['I30'].max() 
                    df_partial_result.at[evento,'RF_KE']=df_event['RF_KE'].sum()
                    df_partial_result.at[evento,'EI30']=EI30
                    df_partial_result.at[evento,'month']=month
                
                else:

                    df_partial_result.at[evento,'rain_mm']=df_rainfall_events.iloc[evento,2]  
                    df_partial_result.at[evento,'duration']=df_rainfall_events.iloc[evento,3]        
                    df_partial_result.at[evento,'I30']=0
                    df_partial_result.at[evento,'RF_KE']=0
                    df_partial_result.at[evento,'EI30']=0
                    df_partial_result.at[evento,'month']=month
                #set final results        
            with pd.HDFStore(general_path + f'/1 - Results_Telemetria/Results_{state}_{year}.h5', 'a') as store:
                for month in range(1, 13):
                #set monthly results
                    df_month = df_partial_result[df_partial_result['month']==month]
                    EI30_M=df_month['EI30'].sum()
                    rain=df_month['rain_mm'].sum()
                    E_D = EI30_M / rain if rain != 0 else float('nan')
                    monthly_results[month]["gauge_code"].append(code)
                    monthly_results[month]["EI30"].append(EI30_M)
                    monthly_results[month]["E_D"].append(E_D)
                    monthly_results[month]["rain"].append(rain)                
                    
                    store.put(f'month_{month:02d}',pd.DataFrame(monthly_results[month]))
            
                #set anual results
                
                EI30_T=df_partial_result['EI30'].sum()
                E_D=EI30_T/df_partial_result['rain_mm'].sum()
                rain=df_partial_result['rain_mm'].sum()
                annual_results["gauge_code"].append(code)
                annual_results["EI30"].append(EI30_T)
                annual_results["E_D"].append(E_D)
                annual_results["rain"].append(rain)
                store.put('annual', pd.DataFrame(annual_results))


            print(f"  -code: {code} , time: {(datetime.now()-start_time_code).total_seconds():.2f}s , {len(annual_results['E_D'])}/{len(df_tel_hq[name_col].dropna())}")

        minutes, seconds = divmod((datetime.now()-start_time).total_seconds(), 60)#total duration
        print(f"{state}_{year} completed, total duration: {int(minutes)}m {seconds:.2f}s")  